Equation (4) from the Touchard paper:

$$\frac{n^2(n-1)}{6}\sigma_n = \sum_{k=1}^{n-1}(3n^2 - 10k^2)\sigma_k\sigma_{n-k}$$

So if we rearrange that equation, we can compute $\sigma_n$ for $n > 1$. The equation to compute $\sigma_n$ is:

$$\sigma_n = \frac{6}{n^2(n-1)} \sum_{k=1}^{n-1}(3n^2 - 10k^2)\sigma_k\sigma_{n-k}$$.

#### First Goal:
Make an optimized function that spits out a sequence of $\sigma_n$.

In [1]:
#From the email I sent you before fall last year


sigma_list = [0,1]
for n in range(2,21):
    sigma_n = 0
    for k in range(1,n):
        sigma_n += (3*(n*n) - 10*(k*k))*sigma_list[k]*sigma_list[n-k]
    sigma_list.append((6*sigma_n)//((n*n)*(n-1)))
print(sigma_list)

[0, 1, 3, 4, 7, 6, 12, 8, 15, 13, 18, 12, 28, 14, 24, 24, 31, 18, 39, 20, 42]


In [1]:
import numpy as np

In [2]:
def sigma_list(n): # To compare to the next function which uses a numpy trick
    sigma_list = [0,1] # Same thing as the last algorithm, just wrapped in a function
    for j in range(2,n+1):
        sigma_n = 0
        for k in range(1,j):
            sigma_k = sigma_list[k]
            sigma_nk = sigma_list[j-k]
            sigma_n += (3*(j*j) - 10*(k*k))*sigma_k*sigma_nk
        sigma_list.append((6*sigma_n)//((j*j)*(j-1)))
    return sigma_list

In [3]:
def sigma_vectorized_2(N):
    sigmalist = np.zeros(N+1)
    sigmalist[1] = 1
    for n in range(2,N+1): #I'm sure I can use vectorization here too, just gotta figure out how
        nklist = 3*np.array([n]*(n-1))**2 - 10*np.arange(1,n)**2
        sigmalist[n] = (6/((n*n)*(n-1)))*np.dot(nklist,sigmalist[1:n]*sigmalist[n-1:0:-1])
    return sigmalist

In [4]:
sigma_list(1000)[1000]

2340

In [5]:
sigma_vectorized_2(1000)[1000]

2339.999999999726

I'm thinking that maybe if I make a 2d version of `nklist` in `sigma_vectorized_2` with prescribed values, and just loop through slices of that instead of making a new 1d array each loop, then maybe it'd speed things up. I'll need to experiment more.

In [6]:
def sigma_vectorized(N):
    sigmalist = np.zeros(N+1)
    sigmalist[1] = 1
    
    nsq_list = np.arange(N+1)**2
    ksq_list = nsq_list
    
    a=3
    c=-10
    
    NK_array = np.add.outer(a*nsq_list,c*ksq_list)
    
    for n in range(2,N+1):
        vec_A = NK_array[n,1:n] * sigmalist[1:n]
        vec_B = sigmalist[n-1:0:-1]
        sigmalist[n] = (6/((n*n)*(n-1)))*np.dot(vec_A, vec_B)
    return sigmalist

In [7]:
sigma_vectorized(20)

array([ 0.,  1.,  3.,  4.,  7.,  6., 12.,  8., 15., 13., 18., 12., 28.,
       14., 24., 24., 31., 18., 39., 20., 42.])

In [10]:
%timeit sigma_vectorized(20)

74.3 µs ± 509 ns per loop (mean ± std. dev. of 7 runs, 10000 loops each)


In [11]:
%timeit sigma_vectorized_2(20)

187 µs ± 432 ns per loop (mean ± std. dev. of 7 runs, 10000 loops each)


In [13]:
%timeit sigma_list(20)

64.1 µs ± 351 ns per loop (mean ± std. dev. of 7 runs, 10000 loops each)


In [44]:
def sigma_augment(N):
    sigmalist = np.zeros(N+1)
    sigmalist[1] = 1
    
    nsq_list = np.arange(N+1)**2
    ksq_list = nsq_list
    
    a=20
    c=-20
    
    NK_array = np.add.outer(a*nsq_list,c*ksq_list)
    
    for n in range(2,N+1):
        vec_A = NK_array[n,1:n] * sigmalist[1:n] # Entrywise multiplication
        vec_B = sigmalist[n-1:0:-1]
        sigmalist[n] = (6/((n*n)*(n-1)))*np.dot(vec_A, vec_B)
    return sigmalist

In [45]:
sigma_augment(20)[3]

7800.0

We want to play with the weights of the binomial in the sum, so we would have something like
$$
S_n = \frac{6}{n^2(n-1)}\sum_{k=1}^{n-1}(an^2 + ck^2)S_{k}S_{n-k}
$$
with $-10 \leq a,c \leq 10$ for now.

In [58]:
def sequence_check():
    S_array = np.zeros((20,20,20))
    for r in range(20):
        for c in range(20):
            S_array[r,c,1] = 1

    a_array = np.arange(-10,10) # This goes from -10 to 9, but the 20x20x20 array sounded nice
    c_array = a_array

    nsq_list = np.arange(20)**2
    ksq_list = nsq_list

    ac_array = np.zeros((20,20,20,20))

    for a in a_array:
        for c in c_array:
            ac_array[a+10,c+10] = np.add.outer(a*nsq_list,c*ksq_list)

    for a in range(20):
        for c in range(20):
            for n in range(2,20):
                vec_A = ac_array[a,c,n,1:n]*S_array[a,c,1:n]
                vec_B = S_array[a,c,n-1:0:-1]
                S_array[a,c,n] = (6/((n*n)*(n-1)))*np.dot(vec_A, vec_B)


    return S_array[13,0]

In [59]:
sequence_check()

array([ 0.,  1.,  3.,  4.,  7.,  6., 12.,  8., 15., 13., 18., 12., 28.,
       14., 24., 24., 31., 18., 39., 20.])

In [57]:
%timeit sequence_check()

24.9 ms ± 357 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)
